In [1]:
from functools import wraps, reduce

import tensorflow.keras.backend as K
from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, Concatenate, MaxPooling2D, BatchNormalization, Activation, UpSampling2D, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.regularizers import l2

#from common.backbones.layers import YoloConv2D, YoloDepthwiseConv2D, CustomBatchNormalization

### 定义 compose 函数
使用Python的Lambda表达式，顺次执行函数列表，且前一个函数的输出是后一个函数的输入。compose函数适用于在神经网络中连接两个层。

In [2]:
def compose(*funcs):
    """Compose arbitrarily many functions, evaluated left to right.

    Reference: https://mathieularose.com/function-composition-in-python/
    """
    if funcs:
        return reduce(lambda f, g: lambda *a, **kw: g(f(*a, **kw)), funcs)
    else:
        raise ValueError('Composition of empty sequence not supported.')

### 定义 Darknet_Conv2D


In [2]:
@wraps(Conv2D)
def Darknet_Conv2D(*args, **kwargs):
    """Wrapper to set Darknet parameters for Convolution2D."""
    darknet_conv_kwargs = {'kernel_regularizer': l2(5e-4)}
    darknet_conv_kwargs['bias_regularizer'] = l2(5e-4)
    darknet_conv_kwargs['padding'] = 'valid' if kwargs.get('strides')==(2,2) else 'same'
    darknet_conv_kwargs.update(kwargs)
    return Conv2D(*args, **darknet_conv_kwargs)

### 定义 Mish 激活函数

In [3]:
def mish(x):
    return x * K.tanh(K.softplus(x))

### 定义 Darknet_CBM

In [5]:
def Darknet_CBM(*args, **kwargs):
    """Darknet Convolution2D followed by BatchNormalization and Mish."""
    no_bias_kwargs = {'use_bias': False}  # 没懂为啥用 no_bias
    no_bias_kwargs.update(kwargs)
    return compose(
        Darknet_Conv2D(*args, **no_bias_kwargs),
        BatchNormalization(),
        Mish()
    )

### 定义 Darknet_CBL

In [17]:
def Darknet_CBL(*args, **kwargs):
    """Darknet Convolution2D followed by BatchNormalization and LeakyReLU."""
    
    no_bias_kwargs = {'use_bias': False}  # 没懂为啥用 no_bias
    no_bias_kwargs.update(kwargs)
    return compose(
        Darknet_Conv2D(*args, **no_bias_kwargs),
        BatchNormalization(),
        LeakyReLU(alpha=0.1)
    )

### 定义 Spp

In [3]:
def Spp(x):
    y1 = MaxPooling2D(pool_size=(5,5), strides=(1,1), padding='same')(x)
    y2 = MaxPooling2D(pool_size=(9,9), strides=(1,1), padding='same')(x)
    y3 = MaxPooling2D(pool_size=(13,13), strides=(1,1), padding='same')(x)
    
    y = Concatenate()([y3, y2, y1, x])
    return y

### 定义 make_five_darknet_CBL

In [ ]:
def make_five_darknet_CBL(x, num_filters):
    # 五次卷积
    x = compose(
            Darknet_Conv2D_BN_Leaky(num_filters, (1,1)),
            Darknet_Conv2D_BN_Leaky(num_filters*2, (3,3)),
            Darknet_Conv2D_BN_Leaky(num_filters, (1,1)),
            Darknet_Conv2D_BN_Leaky(num_filters*2, (3,3)),
            Darknet_Conv2D_BN_Leaky(num_filters, (1,1)))(x)

    return x

### 定义 make_three_darknet_CBL

In [2]:
def make_three_darknet_CBL(x, num_filters):
    # 三次卷积
    x = compose(
            Darknet_Conv2D_BN_Leaky(num_filters, (1,1)),
            Darknet_Conv2D_BN_Leaky(num_filters*2, (3,3)),
            Darknet_Conv2D_BN_Leaky(num_filters, (1,1)))(x)

    return x